In [ ]:
def compute_deformations(mu, kappa, N, max_time, num_time_points=100):
    """
    Compute harmonic deformations of a bladed disk system by solving the eigenproblem.
    
    Args:
        mu: Mass ratio between disk and blades (Md/mb).
        kappa: Stiffness ratio between disk and blades (kd/kb).
        N: Number of blades.
        max_time: Maximum simulation time.
        num_time_points: Number of time points to evaluate.

    Returns:
        A list of deformations for each time step: [t, disk_def, blade_def]
        where disk_def and blade_def are arrays of length N representing deformations at each nodal diameter.
    """
    # Define parameters
    nodal_diameters = np.arange(-N // 2, N // 2 + 1)  # Nodal diameters
    delta_theta = 2 * np.pi / N  # Angular difference between blades
    time = np.linspace(0, max_time, num_time_points)
    
    # Precompute thetas
    thetas = np.linspace(0, 2 * np.pi, N, endpoint=False)

    # Initialize output
    results = []

    # Precompute eigenvalues and eigenvectors for each nodal diameter
    eigen_data = []
    for nodal_diameter in nodal_diameters:
        sin_theta = np.sin(nodal_diameter * delta_theta / 2) ** 2

        # Stiffness matrix K
        K = np.array([
            [1 + 4 * kappa * sin_theta, -1],
            [-1, 1]
        ])

        # Mass matrix M
        M = np.array([
            [mu, 0],
            [0, 1]
        ])

        # Solve the eigenvalue problem using numpy's eig
        w_tidal_squared, xy_hat = np.linalg.eig(np.linalg.inv(M) @ K)
        frequencies = np.sqrt(np.real(w_tidal_squared))  # Only take the real part
        eigen_data.append((nodal_diameter, frequencies, xy_hat))

    # Calculate deformations for each time step
    for t in time:
        disk_def = np.zeros(N)
        blade_def = np.zeros(N)

        for idx, theta in enumerate(thetas):
            for nodal_diameter, frequencies, xy_hat in eigen_data:
                omega_t = frequencies * t

                # Extract disk and blade components
                x_hat = xy_hat[0, :]  # Disk component
                y_hat = xy_hat[1, :]  # Blade component

                # Compute deformations using harmonic solutions
                disk_contrib = np.real(x_hat * np.exp(1j * (nodal_diameter * theta + omega_t)))
                blade_contrib = np.real(y_hat * np.exp(1j * (nodal_diameter * theta + omega_t)))

                disk_def[idx] += np.sum(disk_contrib)
                blade_def[idx] += np.sum(blade_contrib)

        # Append the result as [t, disk_def, blade_def]
        results.append([t, disk_def, blade_def])

    return results

    # Material properties
E = 200e9  # Young's modulus in Pascals (steel)
rho = 7800  # Density in kg/m^3 (steel)
v = 0.3  # Poisson's ratio

# Disk properties
r_disk = 0.1  # Radius of the disk in meters
t_disk = 0.01  # Thickness of the disk in meters

M_disk = rho * np.pi * r_disk**2 * t_disk  # Mass of the disk in kg, mass = density * volume = density * pi * radius^2 * thickness



# Blade properties
blade_num = 12 # Number of blades
#L_blade = 0.1  # Length of the blade in meters
L_blade = 0.05
t_blade = 0.01  # Thickness of the blade in meters
#b_blade = 0.02  # Width of the blade in meters
b_blade = 0.0125

I_blade = (b_blade * t_blade**3) / 12  # Second moment of area for rectangular cross-section
k_blade = 3 * E * I_blade / L_blade**3
M_blade = rho*L_blade*t_blade*b_blade  # Mass of a blade in kg, mass = density * volume = density * length * width * thickness

# Calculate threshold for thin/thick disk and calculate disk stiffness accordingly
thresh = t_disk / r_disk

# Timoshenko beam theory, neglecting shear stress

# Calculate disk stiffness
if thresh <= 0.1:  # Thin disk
    k_disk = 2 * E * t_disk**3 / (3 * (1 - v**2) * r_disk**3)
else:  # Thick disk
    k_disk = E * np.pi * r_disk**2 / t_disk

# Calculate coefficients
kappa = k_disk / k_blade  # Stiffness ratio between disk and blades
mu = M_disk / M_blade #for future use

max_time = 10
timesteps = 100
information = compute_deformations(mu,kappa,blade_num, max_time, timesteps)

# Discretization parameters
# Blade Discretization
num_blade_segments = 5  # Number of segments along the blade length # change to discretization size later

# Compute blade positions
blade_angles = np.linspace(0, 2 * np.pi, blade_num, endpoint=False)
blade_start_x = r_disk * np.cos(blade_angles)
blade_start_y = r_disk * np.sin(blade_angles)
blade_end_x = (r_disk + L_blade) * np.cos(blade_angles)
blade_end_y = (r_disk + L_blade) * np.sin(blade_angles)

# Disk Discretization
num_radial = 10  # Number of radial divisions
#num_circumferential = blade_num *3  # Number of circumferential divisions = number of blades * 3, each blade is connected to a disk segment divided into 3 parts
radial_positions = np.linspace(0, r_disk, num_radial)

disk_deformations = [info[0:2] for info in information]
blade_deformations = [[info[0], info[2]] for info in information]

# Visualization: Disk with blades
# Create the figure
fig, ax = plt.subplots()
cmap = plt.cm.coolwarm
disk_color = np.zeros_like(blade_angles)  # Initialize disk deformation color
initial_deformation = np.zeros_like(blade_angles)
normalized_deformation = (initial_deformation - initial_deformation.min()) / (initial_deformation.max() - initial_deformation.min() + 1e-6)
# Draw Disk (Discretized)
disk_patches = []
for i in range(num_radial - 1):
    for j in range(blade_num):
        #Define vertices for each patch (quadrilateral)
        theta1 = blade_angles[j] + np.pi/4
        theta2 = blade_angles[(j + 1) % blade_num] + np.pi/4
        r1, r2 = radial_positions[i], radial_positions[i + 1]

        x1, y1 = r1 * np.cos(theta1), r1 * np.sin(theta1)
        x2, y2 = r1 * np.cos(theta2), r1 * np.sin(theta2)
        x3, y3 = r2 * np.cos(theta2), r2 * np.sin(theta2)
        x4, y4 = r2 * np.cos(theta1), r2 * np.sin(theta1)

        # Create patch
        patch = Polygon([(x1, y1), (x2, y2), (x3, y3), (x4, y4)], color=cmap(normalized_deformation[j]), alpha=0.6)
        ax.add_patch(patch)
        disk_patches.append(patch)

# Draw blades as rectangles, centered on the angles
# Discretize blades similar to disks
blade_patches = []


for angle in blade_angles:
    for segment in range(num_blade_segments):
        # Compute the start and end positions of the segment
        segment_start = segment * (L_blade / num_blade_segments)
        segment_end = (segment + 1) * (L_blade / num_blade_segments)
        
        # Compute the blade's center position for the segment
        blade_center_x_start = (r_disk + segment_start) * np.cos(angle)
        blade_center_y_start = (r_disk + segment_start) * np.sin(angle)
        blade_center_x_end = (r_disk + segment_end) * np.cos(angle)
        blade_center_y_end = (r_disk + segment_end) * np.sin(angle)
        
        # Define vertices for each patch (quadrilateral)
        x1, y1 = blade_center_x_start - (b_blade / 2) * np.sin(angle), blade_center_y_start + (b_blade / 2) * np.cos(angle)
        x2, y2 = blade_center_x_start + (b_blade / 2) * np.sin(angle), blade_center_y_start - (b_blade / 2) * np.cos(angle)
        x3, y3 = blade_center_x_end + (b_blade / 2) * np.sin(angle), blade_center_y_end - (b_blade / 2) * np.cos(angle)
        x4, y4 = blade_center_x_end - (b_blade / 2) * np.sin(angle), blade_center_y_end + (b_blade / 2) * np.cos(angle)
        
        # Create patch
        patch = Polygon([(x1, y1), (x2, y2), (x3, y3), (x4, y4)], color=cmap(normalized_deformation[j]), alpha=0.6)
        ax.add_patch(patch)
        blade_patches.append(patch)

# Define blades as combination of rectangles
blades = blade_patches

# Set up plot aesthetics
ax.set_aspect('equal')
ax.set_xlim(-r_disk - L_blade - 0.1, r_disk + L_blade + 0.1)
ax.set_ylim(-r_disk - L_blade - 0.1, r_disk + L_blade + 0.1)
ax.axis('off')
fig.colorbar(plt.cm.ScalarMappable(cmap=cmap), ax=ax, label='Normalized Deformation')


def update(frame):
    # Access current frame deformation data
    disk_deformation = disk_deformations[frame][1]
    blade_deformation = blade_deformations[frame][1]
    
    # Normalize deformations for color mapping
    disk_norm = (disk_deformation - np.min(disk_deformation)) / (np.ptp(disk_deformation) + 1e-6) # add 1e-6 to avoid division by zero
    blade_norm = (blade_deformation - np.min(blade_deformation)) / (np.ptp(blade_deformation) + 1e-6)
    
    # Loop through patches and update colors
    for idx, patch in enumerate(disk_patches):
        blade_num = idx % len(disk_norm)  # Ensure the index stays within bounds
        color_value = float(disk_norm[blade_num])
        patch.set_color(cmap(color_value))

    for idx, patch in enumerate(blade_patches):
        blade_num = idx % len(blade_norm)  # Ensure the index stays within bounds
        color_value = float(blade_norm[blade_num])
        patch.set_color(cmap(color_value))

    return disk_patches + blade_patches

# Animate the deformation
ani = FuncAnimation(fig, update, frames=timesteps, blit=True)
plt.close()

HTML(ani.to_jshtml())